# Defencer OLS

### Import Package

In [4]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import MySQLdb
import os

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score

### Connect DB & Get Defencer Player Data

In [5]:
db = MySQLdb.connect(
    os.environ.get("DATABASE_HOST"),
    os.environ.get("DATABASE_USERNAME"),
    os.environ.get("PASSWORD"),
    os.environ.get("DATABASE_NAME"),
    charset='utf8',
)


def make_query(position):
    """
    
    parameter------------
    position : M, D, F, G
    
    return---------------
    SQL_QUERY String
    
    """
    
    SQL_QUERY = """
        SELECT 
            age, tall, weight, apps_start, apps_sub, mins, goals, assists, yel, red
            , spg, ps_x, motm, aw, tackles, inter, fouls, offsides, clear, drb, blocks
            , owng, keyp_x, fouled, off, disp, unstch, avgp, ps_y, rating        
        FROM player
    """
    
    if position == "F":
        SQL_QUERY += """
            WHERE position not like "%,%" and position like "%FW%" and mins > 270
        """
    
    if position == "M":
        SQL_QUERY += """
            WHERE position not like "%,%" and position like "%M%" and mins > 270
        """
    
    if position == "D":
        SQL_QUERY += """
            WHERE position not like "%,%" and position like "%D%" and position not like " DMC"  and mins > 270
        """
    
    if position == "G":
        SQL_QUERY += """
            WHERE position not like "%,%" and position like "%G%" and mins > 270
        """
    
    return SQL_QUERY

# defencer
SQL_QUERY = make_query("D")
defenser_df = pd.read_sql(SQL_QUERY, db)

len(defenser_df)

817

### Scaling 

In [3]:
X = defenser_df.ix[:,:-1]
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)

dfX0 = pd.DataFrame(X_scaled, columns=X.columns)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(defenser_df.ix[:,-1], columns=["rating"])
d_df = pd.concat([dfX, dfy], axis=1)
d_df.head()

,const,age,tall,weight,apps_start,apps_sub,mins,goals,assists,yel,...,blocks,owng,keyp_x,fouled,off,disp,unstch,avgp,ps_y,rating
0,1,5.342523,15.105863,10.954348,2.615560,0.466727,2.632199,2.827191,0.900141,2.688382,...,1.967895,2.427616,0.654447,2.379510,0.000000,0.672855,1.001993,4.325854,11.328504,6.93
1,1,6.799575,14.146761,9.665601,3.452539,0.466727,3.454909,0.000000,2.700424,2.304327,...,0.327983,0.000000,2.290563,1.665657,0.000000,2.018565,2.337983,2.718860,10.961246,6.90
2,1,6.556733,14.466461,9.923350,1.569336,3.733816,1.738155,0.000000,0.000000,0.384055,...,0.327983,0.000000,1.308893,0.951804,1.435429,1.009282,1.669988,3.083691,10.890619,6.89
3,1,6.799575,14.786162,10.181100,2.929427,0.933454,2.954387,0.000000,0.900141,1.920273,...,2.295878,2.427616,0.654447,0.237951,1.435429,0.672855,1.001993,3.274793,11.342629,6.86
4,1,7.285259,14.546387,9.407851,3.347916,0.000000,3.369309,0.942397,3.600565,4.224600,...,0.655965,0.000000,2.945009,2.141559,0.000000,2.691419,3.339975,4.508270,11.653386,7.60


### Summary OLS

In [4]:
model = sm.OLS(d_df.ix[:, -1], d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     92.66
Date:                Thu, 30 Jun 2016   Prob (F-statistic):          1.84e-227
Time:                        12:06:26   Log-Likelihood:                 525.03
No. Observations:                 817   AIC:                            -992.1
Df Residuals:                     788   BIC:                            -855.6
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.4647      0.108     50.706      0.0

### Find Proper Model

In [5]:
# remove features
remove_column_list = [
    "age", "tall", "weight", "apps_start", "apps_sub", "mins", "yel", "unstch"
]
removed_d_df = d_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed_d_df.ix[:, -1], removed_d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     129.1
Date:                Thu, 30 Jun 2016   Prob (F-statistic):          1.06e-233
Time:                        12:10:54   Log-Likelihood:                 520.29
No. Observations:                 817   AIC:                            -998.6
Df Residuals:                     796   BIC:                            -899.8
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.5091      0.073     75.010      0.0

### ANOVA

In [6]:
formula_str = """
rating ~ goals + assists + red + spg + ps_x + motm + aw
+ tackles + inter + fouls + offsides + clear + drb + blocks
+ owng + keyp_x + fouled + off + disp + avgp + ps_y
"""

model = sm.OLS.from_formula(formula_str, data=removed_d_df)
result = model.fit()
table_anova = sm.stats.anova_lm(result)
table_anova

,df,sum_sq,mean_sq,F,PR(>F)
goals,1.0,7.351482,7.351482,437.207013,1.015348e-77
assists,1.0,4.562316,4.562316,271.329858,1.116092e-52
red,1.0,0.018432,0.018432,1.096175,2.954250e-01
spg,1.0,1.880096,1.880096,111.812964,1.524576e-24
ps_x,1.0,2.231002,2.231002,132.682065,1.671015e-28
motm,1.0,7.489474,7.489474,445.413638,7.204196e-79
aw,1.0,4.249837,4.249837,252.746118,1.245196e-49
tackles,1.0,8.439651,8.439651,501.922530,1.403409e-86
inter,1.0,3.188942,3.188942,189.652623,7.383641e-39
fouls,1.0,0.364824,0.364824,21.696824,3.741386e-06


In [7]:
# remove feature 2
remove_column_list = [
    "red", "offsides", "drb", "blocks", "off", "disp", "ps_y"
]
removed2_d_df = removed_d_df.drop(remove_column_list, axis=1) 

model = sm.OLS(removed2_d_df.ix[:, -1], removed2_d_df.ix[:, :-1])
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     163.4
Date:                Thu, 30 Jun 2016   Prob (F-statistic):          1.28e-223
Time:                        12:11:27   Log-Likelihood:                 480.88
No. Observations:                 817   AIC:                            -931.8
Df Residuals:                     802   BIC:                            -861.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          5.4991      0.076     72.405      0.0

### Result

Key Features = tackles, aw, inter, clear